In [1]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
import pandas as pd
import csv
import re
import string

data_frame = pd.read_csv('C:\\Users\\surya\\Downloads\\wiki_raw_and_mapping\\wiki_raw_and_mapping\\input_swap_wiki_50k.tsv', sep='\t')

print(data_frame)

               id                                          sentence1  \
0      swap_00001  Between 1962 and 1967 in Pennsylvania, annual ...   
1      swap_00002  Martina Navratilova/Emilio Sánchez defeated Be...   
2      swap_00003  Desdemona encounters Iago carrying buckets of ...   
3      swap_00004  On 20 November 2010, Sergio got a chance to av...   
4      swap_00005  There, the victory swung with every meter lost...   
...           ...                                                ...   
49995  swap_49996  At Waynesburg it collects a short stream known...   
49996  swap_49997  In the books, Windsong's name is Go For the Bl...   
49997  swap_49998  She then taught for a year in Balham before le...   
49998  swap_49999  Acute mesenteric ischemia was first described ...   
49999  swap_50000  Pajuveski is a village in Haljala Parish, Lään...   

                                               sentence2  
0      Between 1962 and 1967 in Pennsylvania, annual ...  
1      Betsy Nage

In [3]:
import pandas as pd

file_path = 'C:\\Users\\surya\\Downloads\\wiki_raw_and_mapping\\wiki_raw_and_mapping\\input_backtransl_wiki_with_swap_id.tsv'

def read_csv_handle_multiple_errors(file_path, sep='\t'):
    skip_rows = set()
    while True:
        try:
            df = pd.read_csv(file_path, sep=sep, skiprows=list(skip_rows))
            break
        except pd.errors.ParserError as e:
            error_message = str(e)
            try:
                line_number = int(error_message.split('line')[1].split(',')[0]) - 1
                if line_number in skip_rows:
                    break
                skip_rows.add(line_number)
            except Exception as err:
                print("An error occurred while parsing the error message:", err)
                break

    return df
data_frame1 = read_csv_handle_multiple_errors(file_path)

print(data_frame1)


                          id  \
0      backtransl_set3_03491   
1      backtransl_set3_04371   
2      backtransl_set3_03621   
3      backtransl_set3_04952   
4      backtransl_set1_01157   
...                      ...   
26513  backtransl_set1_09460   
26514  backtransl_set1_14350   
26515  backtransl_set2_03959   
26516  backtransl_set2_02524   
26517  backtransl_set2_04849   

                                               sentence1  \
0      He wrote the script in cooperation with Bianca...   
1      He wrote the script in cooperation with Bianca...   
2      He wrote the script in cooperation with Cyril ...   
3      He wrote the script in cooperation with Cyril ...   
4      Most Arabs in France are from the Maghreb but ...   
...                                                  ...   
26513  Abe Drexler (Charlie Hofheimer) calls Elisabet...   
26514  Abe Drexler (Charlie Hofheimer) calls Elisabet...   
26515  The Military ranks of Tajikistan are the milit...   
26516  The mili

In [6]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s\.,!?]', '', text)
    PUNCT_TO_REMOVE = string.punctuation
    clean_text = text.translate(str.maketrans("", "", PUNCT_TO_REMOVE))
    clean_text = re.sub(r'\s+', ' ', clean_text).strip()
    return clean_text

In [5]:
data_frame.columns
data_frame.dtypes

id           object
sentence1    object
sentence2    object
dtype: object

In [6]:
data_frame1.columns
data_frame1.dtypes

id           object
sentence1    object
sentence2    object
swap_id      object
dtype: object

In [7]:
data_frame['id'] = data_frame['id'].astype(str)
data_frame['sentence1'] = data_frame['sentence1'].astype(str)
data_frame['sentence2'] = data_frame['sentence2'].astype(str)
print(data_frame.dtypes)

id           object
sentence1    object
sentence2    object
dtype: object


In [8]:
data_frame1['id'] = data_frame1['id'].astype(str)
data_frame1['sentence1'] = data_frame1['sentence1'].astype(str)
data_frame1['sentence2'] = data_frame1['sentence2'].astype(str)
data_frame1['swap_id'] = data_frame1['swap_id'].astype(str)
print(data_frame1.dtypes)

id           object
sentence1    object
sentence2    object
swap_id      object
dtype: object


In [4]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [11]:
formatted_data = []

for index, row in data_frame.iterrows():
    input_text = clean_text(row['sentence1'])
    target_text = clean_text(row['sentence2'])
    formatted_data.append({'input': input_text, 'target': target_text})

for index, row in data_frame1.iterrows():
    input_text = clean_text(row['sentence1'])
    target_text = clean_text(row['sentence2'])
    formatted_data.append({'input': input_text, 'target': target_text})

print(len(formatted_data))
print(formatted_data[:10])

76518
[{'input': 'between 1962 and 1967 in pennsylvania annual survival of breeding eastern towhees calculated from mistnetting recaptures was 58', 'target': 'between 1962 and 1967 in pennsylvania annual survival of breeding mistnetting towhees calculated from eastern recaptures was 58'}, {'input': 'martina navratilovaemilio sánchez defeated betsy nagelsenpaul annacone 6 4 6 7 7 6', 'target': 'betsy nagelsenpaul annacone defeated martina navratilovaemilio sánchez 6 4 6 7 7 6'}, {'input': 'desdemona encounters iago carrying buckets of filth and iago stirs jealousy in her', 'target': 'iago encounters desdemona carrying buckets of filth and iago stirs jealousy in her'}, {'input': 'on 20 november 2010 sergio got a chance to avenge his loss to williams', 'target': 'on 20 november 2010 williams got a chance to avenge his loss to sergio'}, {'input': 'there the victory swung with every meter lost or gained and where machetes and yataganes created a slaughter between both sides', 'target': 'the

In [14]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
model = T5ForConditionalGeneration.from_pretrained('google-t5/t5-small').to(device)
tokenizer = T5Tokenizer.from_pretrained('google-t5/t5-small')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
formatted_data_out = pd.DataFrame(formatted_data)
formatted_data_out.to_csv('C:\\Users\\surya\\Downloads\\wiki_raw_and_mapping\\wiki_raw_and_mapping\\train_data.txt', sep='\t', index=False, header=True)
print("Formatted data stored in train_data.txt")

Formatted data stored in train_data.txt


In [16]:
formatted_dataset = {
    "input_text": [pair["input"] for pair in formatted_data],
    "output_text": [pair["target"] for pair in formatted_data]
}

In [16]:
def tokenize_function(talks):
    tokenized_inputs = tokenizer(talks['input_text'], padding='max_length', truncation=True, max_length=256)
    tokenized_outputs = tokenizer(talks['output_text'], padding='max_length', truncation=True, max_length=256)
    return {'input_ids': tokenized_inputs['input_ids'], 'attention_mask': tokenized_inputs['attention_mask'], 'labels': tokenized_outputs['input_ids']}

In [18]:
from datasets import Dataset
dataset = Dataset.from_dict(formatted_dataset)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

train_eval_split = tokenized_datasets.train_test_split(test_size=0.2, shuffle=False)
train_dataset = train_eval_split['train']
eval_dataset = train_eval_split['test']

val_test_split = eval_dataset.train_test_split(test_size = 0.5, shuffle=False)
val_dataset = val_test_split['train']
test_dataset = val_test_split['test']

Map:   0%|          | 0/76518 [00:00<?, ? examples/s]

In [19]:
train_dataset[:10]

{'input_text': ['between 1962 and 1967 in pennsylvania annual survival of breeding eastern towhees calculated from mistnetting recaptures was 58',
  'martina navratilovaemilio sánchez defeated betsy nagelsenpaul annacone 6 4 6 7 7 6',
  'desdemona encounters iago carrying buckets of filth and iago stirs jealousy in her',
  'on 20 november 2010 sergio got a chance to avenge his loss to williams',
  'there the victory swung with every meter lost or gained and where machetes and yataganes created a slaughter between both sides',
  'la vegueta is a village in tinajo las palmas province of western lanzarote in the canary islands',
  'it is found in the eastern andes of northwestern argentina and southern bolivia',
  'razer punctured pussycats body and lifted it with the arm but pussycat fell and escaped',
  'corticons competitors include bosch software innovations fico ibmilog inrule technology openrules pegasystems red hat and others',
  'he was a lifelong friend of charles lutwidge dodgso

In [20]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from torch.optim import AdamW

training_args = Seq2SeqTrainingArguments(
    output_dir='C:\\Users\\surya\\Downloads\\wiki_raw_and_mapping\\wiki_raw_and_mapping\\results',
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='C:\\Users\\surya\\Downloads\\wiki_raw_and_mapping\\wiki_raw_and_mapping\\logs',
    save_strategy='epoch',
    evaluation_strategy='epoch',
    learning_rate=2e-4
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

In [21]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.039100,0.068110
2,0.031400,0.066959
3,0.027100,0.065577
4,0.024600,0.065257
5,0.022400,0.065648


TrainOutput(global_step=19130, training_loss=0.08009476110814319, metrics={'train_runtime': 8264.5161, 'train_samples_per_second': 37.034, 'train_steps_per_second': 2.315, 'total_flos': 2.071203258826752e+16, 'train_loss': 0.08009476110814319, 'epoch': 5.0})

In [22]:
evaluation_metrics = trainer.evaluate()
print(evaluation_metrics["eval_loss"])

0.06564792990684509


In [23]:
tokenizer.save_pretrained('C:\\Users\\surya\\Downloads\\wiki_raw_and_mapping\\wiki_raw_and_mapping\\results\\t5\\tokenizer')

('C:\\Users\\surya\\Downloads\\wiki_raw_and_mapping\\wiki_raw_and_mapping\\results\\t5\\tokenizer\\tokenizer_config.json',
 'C:\\Users\\surya\\Downloads\\wiki_raw_and_mapping\\wiki_raw_and_mapping\\results\\t5\\tokenizer\\special_tokens_map.json',
 'C:\\Users\\surya\\Downloads\\wiki_raw_and_mapping\\wiki_raw_and_mapping\\results\\t5\\tokenizer\\spiece.model',
 'C:\\Users\\surya\\Downloads\\wiki_raw_and_mapping\\wiki_raw_and_mapping\\results\\t5\\tokenizer\\added_tokens.json')

In [24]:
trainer.save_model('C:\\Users\\surya\\Downloads\\wiki_raw_and_mapping\\wiki_raw_and_mapping\\results\\t5\\model')

In [4]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
model_path = 'C:\\Users\\surya\\Downloads\\wiki_raw_and_mapping\\wiki_raw_and_mapping\\results\\t5\\model'
tokenizer_path = 'C:\\Users\\surya\\Downloads\\wiki_raw_and_mapping\\wiki_raw_and_mapping\\results\\t5\\tokenizer'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = T5ForConditionalGeneration.from_pretrained(model_path).to(device)
tokenizer = T5Tokenizer.from_pretrained(tokenizer_path)

model.eval()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [42]:
def generate_text(prompt_text, model, tokenizer, device):

    input_ids = tokenizer(prompt_text, return_tensors='pt', padding="max_length", max_length=256, truncation=True)

    input_ids = input_ids.to(device)

    outputs = model.generate(input_ids=input_ids['input_ids'], max_length = 50, do_sample=True, num_beams=5, no_repeat_ngram_size=2)
    
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return generated_text

prompt_text = "I need you"
generated_text = generate_text(prompt_text, model, tokenizer, device)
print("Generated text:", generated_text)


Generated text: i need you


In [5]:
from huggingface_hub import notebook_login
notebook_login()

In [6]:
model.push_to_hub("kssumanth6/t5_small_sentence_polishing_generator_v2")

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

C:\Users\surya\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\surya\.cache\huggingface\hub\models--kssumanth6--t5_small_sentence_polishing_generator_v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


CommitInfo(commit_url='https://huggingface.co/kssumanth6/t5_small_sentence_polishing_generator_v2/commit/f47e61b652e9747750b0fc4f5603c2599de6f1c2', commit_message='Upload T5ForConditionalGeneration', commit_description='', oid='f47e61b652e9747750b0fc4f5603c2599de6f1c2', pr_url=None, pr_revision=None, pr_num=None)

In [7]:
tokenizer.push_to_hub("kssumanth6/t5_small_sentence_polishing_generator_v2")

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/kssumanth6/t5_small_sentence_polishing_generator_v2/commit/d7a689a547d3d67a77cbf516fbea2a0150779d07', commit_message='Upload tokenizer', commit_description='', oid='d7a689a547d3d67a77cbf516fbea2a0150779d07', pr_url=None, pr_revision=None, pr_num=None)

In [5]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
model = T5ForConditionalGeneration.from_pretrained('kssumanth6/t5_small_sentence_polishing_generator_v2').to(device)
tokenizer = T5Tokenizer.from_pretrained('kssumanth6/t5_small_sentence_polishing_generator_v2')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
import pandas as pd
import csv
import re
import string

data_frame2 = pd.read_csv('C:\\Users\\surya\\Downloads\\wiki_raw_and_mapping\\wiki_raw_and_mapping\\jfleg.csv')

print(data_frame2)

                                                 input  \
0    So I think we can not live if old people could...   
1                                   For not use car .    
2    Here was no promise of morning except that we ...   
3    Thus even today sex is considered as the least...   
4    image you salf you are wark in factory just to...   
..                                                 ...   
749  Other tourists would teach you some tips for t...   
750  The government also should try to reduce the s...   
751  Alot of memories with enogh time to remember w...   
752           Sceene of violence can affect on them .    
753  While the communities in general have reckoned...   

                                                target  
0    So I think we would not be alive if our ancest...  
1                            Not for use with a car .   
2    Here was no promise of morning , except that w...  
3    Thus , even today , sex is considered as the l...  
4    Imagine yours

In [11]:
data_frame2.columns
data_frame2.dtypes

input     object
target    object
dtype: object

In [12]:
data_frame2['input'] = data_frame2['input'].astype(str)
data_frame2['target'] = data_frame2['target'].astype(str)
print(data_frame2.dtypes)

input     object
target    object
dtype: object


In [14]:
formatted_data = []
for index, row in data_frame2.iterrows():
    input_text = clean_text(row['input'])
    target_text = clean_text(row['target'])
    formatted_data.append({'input': input_text, 'target': target_text})

print(len(formatted_data))
print(formatted_data[:10])

754
[{'input': 'so i think we can not live if old people could not find siences and tecnologies and they did not developped', 'target': 'so i think we would not be alive if our ancestors did not develop sciences and technologies'}, {'input': 'for not use car', 'target': 'not for use with a car'}, {'input': 'here was no promise of morning except that we looked up through the trees we saw how low the forest had swung', 'target': 'here was no promise of morning except that we looked up through the trees and we saw how low the forest had swung'}, {'input': 'thus even today sex is considered as the least important topic in many parts of india', 'target': 'thus even today sex is considered as the least important topic in may parts of india'}, {'input': 'image you salf you are wark in factory just to do one thing like pot taire on car if they fire you you will destroy becouse u dont know more than pot taire in car', 'target': 'imagine yourself you are working in factory just to do one thing l

In [15]:
formatted_dataset = {
    "input_text": [pair["input"] for pair in formatted_data],
    "output_text": [pair["target"] for pair in formatted_data]
}

In [18]:
from datasets import Dataset
dataset = Dataset.from_dict(formatted_dataset)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

train_eval_split = tokenized_datasets.train_test_split(test_size=0.1, shuffle=False)
train_dataset = train_eval_split['train']
eval_dataset = train_eval_split['test']

Map:   0%|          | 0/754 [00:00<?, ? examples/s]

In [22]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from torch.optim import AdamW

training_args = Seq2SeqTrainingArguments(
    output_dir='C:\\Users\\surya\\Downloads\\wiki_raw_and_mapping\\wiki_raw_and_mapping\\results',
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='C:\\Users\\surya\\Downloads\\wiki_raw_and_mapping\\wiki_raw_and_mapping\\logs',
    save_strategy='epoch',
    evaluation_strategy='epoch',
    learning_rate=2e-4
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer
)

In [23]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,0.082340
2,No log,0.070025
3,No log,0.063933
4,No log,0.061143
5,No log,0.059408
6,No log,0.057545
7,No log,0.057079
8,No log,0.056613
9,No log,0.055648
10,No log,0.055753


TrainOutput(global_step=430, training_loss=0.07049347633539244, metrics={'train_runtime': 628.771, 'train_samples_per_second': 10.783, 'train_steps_per_second': 0.684, 'total_flos': 458808706990080.0, 'train_loss': 0.07049347633539244, 'epoch': 10.0})

In [24]:
evaluation_metrics = trainer.evaluate()
print(evaluation_metrics["eval_loss"])

0.055752795189619064


In [37]:
def generate_text(prompt_text, model, tokenizer, device):

    input_ids = tokenizer(prompt_text, return_tensors='pt', padding="max_length", max_length=256, truncation=True)

    input_ids = input_ids.to(device)

    outputs = model.generate(input_ids=input_ids['input_ids'], max_length=300, do_sample=True, num_beams=5, no_repeat_ngram_size=2)
    
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return generated_text

prompt_text = "Hello Again !!!"
generated_text = generate_text(prompt_text, model, tokenizer, device)
print("Generated text:", generated_text)


Generated text: Hello again!!!


In [38]:
from huggingface_hub import notebook_login
notebook_login()

In [39]:
tokenizer.save_pretrained('C:\\Users\\surya\\Downloads\\wiki_raw_and_mapping\\wiki_raw_and_mapping\\results\\t5\\tokenizer')

('C:\\Users\\surya\\Downloads\\wiki_raw_and_mapping\\wiki_raw_and_mapping\\results\\t5\\tokenizer\\tokenizer_config.json',
 'C:\\Users\\surya\\Downloads\\wiki_raw_and_mapping\\wiki_raw_and_mapping\\results\\t5\\tokenizer\\special_tokens_map.json',
 'C:\\Users\\surya\\Downloads\\wiki_raw_and_mapping\\wiki_raw_and_mapping\\results\\t5\\tokenizer\\spiece.model',
 'C:\\Users\\surya\\Downloads\\wiki_raw_and_mapping\\wiki_raw_and_mapping\\results\\t5\\tokenizer\\added_tokens.json')

In [40]:
trainer.save_model('C:\\Users\\surya\\Downloads\\wiki_raw_and_mapping\\wiki_raw_and_mapping\\results\\t5\\model')

In [41]:
model.push_to_hub("kssumanth6/t5_small_sentence_polishing_generator_v3")

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/kssumanth6/t5_small_sentence_polishing_generator_v3/commit/fa24a0c0383c0a933eb974e20a0e5bbb85ff4c40', commit_message='Upload T5ForConditionalGeneration', commit_description='', oid='fa24a0c0383c0a933eb974e20a0e5bbb85ff4c40', pr_url=None, pr_revision=None, pr_num=None)

In [42]:
tokenizer.push_to_hub("kssumanth6/t5_small_sentence_polishing_generator_v3")

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

C:\Users\surya\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\surya\.cache\huggingface\hub\models--kssumanth6--t5_small_sentence_polishing_generator_v3. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/kssumanth6/t5_small_sentence_polishing_generator_v3/commit/cce07eacf1cb53706ba3ddfe234036b10b4a6de2', commit_message='Upload tokenizer', commit_description='', oid='cce07eacf1cb53706ba3ddfe234036b10b4a6de2', pr_url=None, pr_revision=None, pr_num=None)